In [1]:
from sklearn.datasets import fetch_openml


In [2]:
mnist= fetch_openml('mnist_784')

In [3]:
x,y = mnist['data'],mnist['target']

In [4]:
x.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
y

0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: class, Length: 70000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [6]:
type(y)

pandas.core.series.Series

In [7]:
x = x.to_numpy()

In [8]:
y = y.to_numpy()

In [9]:
type(y[5])

str

In [10]:
import numpy as np 
y = y.astype(np.int8)

## Preprocessing 

In [11]:
x_train,y_train = x[:60000],y[:60000]
x_test,y_test = x[:10000],y[:10000]

In [12]:
x_train.shape

(60000, 784)

In [13]:
#shuffle training set. 


In [14]:
shuffle =np.random.permutation(60000)
x_train,y_train = x_train[shuffle],y_train[shuffle]

In [15]:
set(y)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [16]:
#normalise x labels 

In [17]:
x_train = x_train/255.0
x_test = x_test/255.0

## Create a model

In [18]:
x_train[0].shape

(784,)

In [19]:
y[0]

5

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
model = Sequential([
    Dense(128,activation='relu',input_shape=(784,)),
    Dense(128,activation='relu'),
    Dense(10,activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Compile the model - specify loss 

In [21]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)


## Perfrom Cross Validation and Train the Model

In [22]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [28]:
kf = KFold(n_splits=5)
accuracies = []

for train_index,test_index in kf.split(x_train):
    x_train_fold,x_val_fold = x_train[train_index],x_train[test_index]
    y_train_fold,y_val_fold = y_train[train_index],y_train[test_index]
     
    model.fit(x_train_fold,y_train_fold)
    y_pred = model.predict(x_val_fold)
    y_predicted = np.argmax(y_pred,axis=1)
    
    accuracy = accuracy_score(y_val_fold,y_predicted)
    accuracies.append(accuracy)
    
mean_accuracy = sum(accuracies) / len(accuracies)
print("Mean accuracy: ", mean_accuracy)   

375/375 [==============================] - 1s 2ms/step
Mean accuracy:  0.9799
